## Example full process

1. Get parameters
1. Create project board and get id
1. Create columns in project board
1. Create issues and get ids
1. Create cards with issues in project board

In [55]:
# Get parameters
import os
from requests import request
import json
import glob
import time

# GitHub authentication
USERNAME = os.environ["GITHUB_USER"]
TOKEN = os.environ["GITHUB_TOKEN"]

# The repository to add this issue to
REPO_OWNER = USERNAME
REPO_NAME = "test"

headers = {"Authorization": f"token {TOKEN}",
            "Accept": "application/vnd.github.v3+json",
        } 

In [54]:
def convert_md(file):
    """[summary]

    Parameters
    ----------
    file : [type]
        [description]

    Returns
    -------
    [type]
        [description]
    """


    with open(file, 'r') as f:
        text = f.read()

        # Get title
        title = text.split("\n", 2)[:1]
        title = title[0].replace("# ", "")

        # Remove title from content and store
        contents = text.split("\n", 2)[2]
    
    return title, contents

def get_issues(repo_owner, repo_name, titles):
    id_issues = []
    url_get_issues = f"https://api.github.com/repos/{repo_owner}/{repo_name}/issues"
    r = request("GET", url_get_issues, headers=headers)
    issues = json.loads(r.content)

    for issue in issues:
        if issue["title"] in titles:
            id_issues.append(issue["id"])

    return id_issues



In [60]:
# Create project board
url_project = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/projects"

data = {"name": "FAIR Software"}
payload = json.dumps(data)
r = request("POST", url=url_project, data=payload, headers=headers)
id_project = json.loads(r.content)["id"]


# Create columns
url_columns = f"https://api.github.com/projects/{id_project}/columns"
columns = ["To do", "In progress", "Done"]

for column in columns:
    data = {"name": column}
    payload = json.dumps(data)
    r = request("POST", url=url_columns, data=payload, headers=headers)
    if column == "To do":
        id_column = json.loads(r.content)["id"]


# Create issues
url_post_issues = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/import/issues"
titles = []
files = glob.glob("../issues/*.md")
labels = ["documentation", "DCC Support"]

for file in files:
    [title, contents] = convert_md(file)
    titles.append(title)
    data = {
        "issue": {
            "title": title,
            "body": contents,
            "closed": False,
            "labels": labels
        }
    }
    payload = json.dumps(data)
    r = request("POST", url=url_post_issues, data=payload, headers=headers)
    assert r.status_code == 202, f"Could not upload issue '{title}'"


# Get issue id's
while len(id_issues) != len(titles):
    time.sleep(1)
    id_issues = get_issues(repo_owner=REPO_OWNER, repo_name=REPO_NAME, titles=titles)

# Move issues
url_cards = f"https://api.github.com/projects/columns/{id_column}/cards"
for id in id_issues:
    data = {"content_type":"Issue", "content_id": id}
    payload = json.dumps(data)
    r = request("POST", url=url_cards, data=payload, headers=headers)


In [58]:
r.status_code


202

In [53]:
len(issues)

5

In [41]:
url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/issues"

response = request("GET", url, headers=headers)
issues = json.loads(response.content)

titles = []
for file in files:
    [title, contents] = convert_md(file)
    titles.append(title)


issue_ids = []
# Collect issue id's
for issue in issues:
    if issue["title"] in titles:
        issue_ids.append(issue["id"])


print(issue_ids)

[1086282150, 1086282147, 1086282144]


In [34]:
# Move issues
url_cards = f"https://api.github.com/projects/columns/{id_column}/cards"
for i in id_issues:
    data = {"content_type":"Issue", "content_id": i}
    payload = json.dumps(data)
    r = request("POST", url=url_cards, data=payload, headers=headers)
    print(r.content)

b'{"message":"Validation Failed","errors":[{"resource":"ProjectCard","code":"unprocessable","field":"data","message":"Could not resolve to a node with the global id of \'MDU6SXNzdWU0NDUzMDM4\'."}],"documentation_url":"https://docs.github.com/v3/projects/cards/#create-a-project-card"}'
b'{"message":"Validation Failed","errors":[{"resource":"ProjectCard","code":"unprocessable","field":"data","message":"contentID must refer to an issue or pull request in the same repository as the project"}],"documentation_url":"https://docs.github.com/v3/projects/cards/#create-a-project-card"}'
b'{"message":"Validation Failed","errors":[{"resource":"ProjectCard","code":"unprocessable","field":"data","message":"Could not resolve to a node with the global id of \'MDU6SXNzdWU0NDUzMDQw\'."}],"documentation_url":"https://docs.github.com/v3/projects/cards/#create-a-project-card"}'


Create a new project

In [8]:
def create_project(project_name="FAIR software"):

    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/projects"

    headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

    data = {"name": project_name}
    payload = json.dumps(data)
    post_request(url, payload, headers)




def post_request(url, payload, headers):
    """Post request to GitHub

    Parameters
    ----------
    url : str
        [description]
    payload : json
        [description]
    headers : dict
        [description]
    """

    response = request("POST", url, data=payload, headers=headers)
    if response.status_code == 201:
        print('Successfully posted the request') 
    else:
        print('Could post the request')
        print("Response:", response.content)




In [9]:
# Create project
create_project()

Successfully posted the request


### Retrieve info from GitHub

In [ ]:
def get_issues():
    """[summary]

    Returns
    -------
    [type]
        [description]
    """
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/issues"

    headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

    response = request("GET", url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.content)
    else:
        return None


def get_projects():
    """[summary]

    Returns
    -------
    [type]
        [description]
    """
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/projects"

    headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

    response = request("GET", url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.content)
    else:        
        return None

def get_request(type):
    """[summary]

    Parameters
    ----------
    type : [type]
        [description]

    Returns
    -------
    [type]
        [description]
    """


    assert type in ["projects", "issues"], "Unrecognized type, input can be 'projects' or 'issues'"
    if type == "project":
        url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/projects"
    elif type == "issues":
        url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/issues"


    headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

    response = request("GET", url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.content)
    else:        
        return None



In [ ]:
issues = get_issues()

projects = get_projects()

print(projects)

# Find project ID
for project in projects:

    # Find project id
    if project["name"] == "FAIR software":
        project_id = project["id"]
        print(project_id)




Create columns in a project

In [ ]:
url = f"https://api.github.com/projects/{project_id}/columns"
headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }
columns = ["To do", "In progress", "Done"]

for column in columns:
    data = {"name": column}
    payload = json.dumps(data)
    post_request(url, payload, headers) 



### Find IDs for columns

In [38]:
# Find all column id's in project board
columns_url = f"https://api.github.com/projects/{id_project}/columns"
headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

response = request("GET", columns_url, headers=headers)
if response.status_code == 200:
    columns = json.loads(response.content)

# print(columns)

for column in columns:
    print(column["name"], ":", column["id"])
    if column["name"] == "To do":
        column_id = column["id"]

To do : 17298006
In progress : 17298007
Done : 17298008


### Add issues to column

In [ ]:
url = f"https://api.github.com/projects/columns/{column_id}/cards"
headers = {
        "Authorization": f"token {TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

issues = get_issues()

issue_names = ["First issue", "Second issue"]
issue_ids = []
# Collect issue id's
for issue in issues:
    if issue["title"] in issue_names:
        issue_ids.append(issue["id"])


for id in issue_ids:
    data = {"content_type":"Issue", "content_id": id}
    payload = json.dumps(data)
    post_request(url, payload, headers)


In [39]:
issues = get_issues()

NameError: name 'get_issues' is not defined